<a href="https://colab.research.google.com/github/IFuentesSR/red_edge_vegetation/blob/main/sampling_Sentinel_Ameriflux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='xxxx') #set your proyect

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [5]:
cloudProbabilityThreshold = 40


def getS2_CLOUD_PROBABILITY(geo):
    # it may also be used the "COPERNICUS/S2_SR_HARMONIZED" collection
    innerJoined = ee.Join.inner().apply(primary=ee.ImageCollection("COPERNICUS/S2_HARMONIZED").filterBounds(geo).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 80)),
                                        secondary=ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY").filterBounds(geo),
                                        condition=ee.Filter.equals(leftField='system:index',
                                                                   rightField='system:index'))
    def mergeImageBands(joinResult):
        return ee.Image(joinResult.get('primary')).addBands(joinResult.get('secondary'))

    newCollection = innerJoined.map(mergeImageBands)
    return ee.ImageCollection(newCollection)


def maskClouds(_img):
    props = _img.propertyNames()
    cloudMask = _img.select('probability').gt(cloudProbabilityThreshold)
    return _img.addBands(cloudMask.rename('clouds')).copyProperties(_img, props)


def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    CLD_PRJ_DIST = 1
    NIR_DRK_THRESH = 0.15
    clouds = ee.Image(maskClouds(img)).select('clouds')
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (clouds.directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10) \
        .reproject(crs=img.select(0).projection(), scale=100) \
        .select('distance') \
        .mask() \
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))


def masking(img):
  return img.updateMask(img.select('probability').gt(cloudProbabilityThreshold).Not()).updateMask(img.select('shadows').Not())


def addNDVI(img):
    return img.addBands(img.normalizedDifference(['B8', 'B4']).rename('NDVI')) \
    .addBands(img.select('B8').divide(img.select('B5')).rename('VREI')) \
    .addBands(img.normalizedDifference(['B8', 'B5']).rename('NDRE'))


def set_date(img):
    return img.set('date', ee.Date(img.get('system:time_start')).format('YYYY-MM-dd'))

In [6]:
table = ee.FeatureCollection('projects/upbeat-imprint-269809/assets/sites_filtered') # Ameriflux stations

In [ ]:
fea = table.first()
name_sta = fea.get('Name')
geo = fea.geometry()
s2 = getS2_CLOUD_PROBABILITY(geo)
s2_shdw = s2.map(add_shadow_bands)
s2_masked = s2_shdw.map(masking)
NDVI_coll = s2_masked.map(addNDVI)
print(NDVI_coll.first().select(['NDVI', 'VREI', 'NDRE']).reduceRegion('mean', geo.buffer(30), 20).getInfo())


{'NDRE': 0.04241167081367253, 'NDVI': 0.1223383970256262, 'VREI': 1.0888338258462893}


In [7]:
def get_data(fea):
    name_sta = fea.get('Name')
    geo = fea.geometry()
    s2 = getS2_CLOUD_PROBABILITY(geo)
    s2_shdw = s2.map(add_shadow_bands)
    s2_masked = s2_shdw.map(masking)
    NDVI_coll = s2_masked.map(addNDVI)
    def inner(img):
        date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')
        values = img.select(['NDVI', 'VREI', 'NDRE']).reduceRegion('mean', geo.buffer(30), 20)
        return ee.Feature(None, {'name':name_sta, 'NDVI':values.get('NDVI'), 'VREI':values.get('VREI'), 'NDRE':values.get('NDRE'), 'date':date, 'system:time_start':ee.Date(img.get('system:time_start')).millis()})
    data = NDVI_coll.map(inner)
    return data


In [8]:
toDrive = table.map(get_data)
ee.batch.Export.table.toDrive(toDrive.flatten(), 'ameriflux_Sentinel1').start()